User info I extracted:  
(data dictionary: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet)
* 'id'
* 'name'
* 'screen_name'
* 'location'
* 'created_at'
* 'followers_count'
* 'friends_count'      (
* 'statuses_count'     (The total number of tweets that the account has posted.)
* 'favourites_count'   (The total number of tweets that the account has liked.)


# Set up your PostgreSQL server
1.    Download and install Postgres.app from https://postgresapp.com/
2.   Open Postgres.app, and click on the database titled with your system user name; a command-line interface will appear.
3.    Create a table named "twitter" by entering CREATE DATABASE twitter;
4.   Execute the code below to extract the records. Ensure that the files "corona-out-3" and "corona-out-3" are placed in the same working directory.

In [1]:
# Establish a connection to the PostgreSQL server

import psycopg2
import json

conn = psycopg2.connect(
    dbname="twitter",
    user="liz",  # Only replace with your system user name
    password="",
    host="localhost",
    port="5432")

# Open a cursor to perform database operations
cur = conn.cursor()    
    
print("PostgreSQL server information")
print(conn.get_dsn_parameters(), "\n")

PostgreSQL server information
{'user': 'liz', 'passfile': '/Users/liz/.pgpass', 'channel_binding': 'prefer', 'dbname': 'twitter', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [2]:
# Create the User table if it doesn't already exist
cur.execute('''
    CREATE TABLE IF NOT EXISTS TwitterUser (
        id BIGINT PRIMARY KEY,
        name TEXT,
        screen_name TEXT,
        location TEXT,
        created_at TIMESTAMP,
        followers_count INTEGER,
        friends_count INTEGER,
        statuses_count INTEGER,
        favourites_count INTEGER
    );
''')
conn.commit()  

In [3]:
def view_users(conn):
        try:
            cur.execute("SELECT * FROM TwitterUser;")
            users = cur.fetchall()
            if len(users)==0:
                print("No data")
            print("Number of user IDs:",len(users))
        except Exception as e:
            print(f"An error occurred: {e}")
            conn.rollback()

# Use this function to view the data in the TwitterUser table
view_users(conn)

No data
Number of user IDs: 0


In [4]:
import psycopg2
import json

def insert_user(cur, user_info):
        try:
            # Check if the user already exists in the database
            cur.execute("SELECT id FROM TwitterUser WHERE id = %s;", (user_info["id"],))
            if cur.fetchone() is None:
                # If the user does not exist, insert their information into the TwitterUser table
                cur.execute("""
                    INSERT INTO TwitterUser (id, name, screen_name, location, created_at, followers_count, friends_count, statuses_count, favourites_count)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
                """, (user_info["id"], user_info["name"], user_info["screen_name"], user_info["location"], user_info["created_at"], user_info["followers_count"], user_info["friends_count"], user_info["statuses_count"], user_info["favourites_count"]))
                conn.commit()
        except Exception as e:
            print(f"An error occurred: {e}")
            conn.rollback()

def insert_records(file):
    with open(file, "r") as f1:
        for line in f1:
            try:
                tweet = json.loads(line)

                # Extract the user information from the tweet
                user_info = tweet["user"]
                # Insert the user information into the User table
                insert_user(cur, user_info)

                # Check if the tweet is a retweet
                if "retweeted_status" in tweet:
                    # Extract the original tweet's user information
                    original_user_info = tweet["retweeted_status"]["user"]
                    # Insert the original tweet's user information into the User table
                    insert_user(cur, original_user_info)

            except:
                # if there is an error loading the json of the tweet, skip
                continue


In [5]:
insert_records("corona-out-2")

In [6]:
view_users(conn)

Number of user IDs: 18461


In [15]:
# cur.execute("SELECT * FROM TwitterUser;")
# users=cur.fetchall()
# for people in users:
#     print(people)

In [7]:
insert_records("corona-out-3")

In [8]:
view_users(conn)

Number of user IDs: 105181


In [10]:
# Transform the data into a DataFrame for improved visualization and readability. 

import pandas as pd
from sqlalchemy import create_engine

# Create an SQLAlchemy engine using the psycopg2 connection string
engine = create_engine("postgresql+psycopg2://liz:@localhost:5432/twitter")

def get_twitter_users_dataframe(engine):
    query = "SELECT * FROM TwitterUser;"
    df = pd.read_sql_query(query, engine)
    return df

# Fetch the data and convert it into a DataFrame
user_df = get_twitter_users_dataframe(engine)


In [11]:
user_df.head(20)

,id,name,screen_name,location,created_at,followers_count,friends_count,statuses_count,favourites_count
0,1242817830946508801,juwelz v,juwelz_v,"Lower East Side, Manhattan",2020-03-25 14:17:28,43,118,906,722
1,16144221,NUFF,nuffsaidny,None,2008-09-05 14:28:41,17112,1515,2599,15790
2,1225145123920588805,efe09,efe0927183508,None,2020-02-05 19:52:38,653,983,4177,1255
3,1087735689091928064,Karanfil Lale,lale_karanfil,None,2019-01-22 15:36:12,897,1120,7986,2776
4,101007632,Ravin Gupta,IamRaavin,india,2010-01-01 16:24:24,499,537,4038,4342
5,268218622,Umesh Agrawal,umesh_agr,None,2011-03-18 10:53:11,288,322,258,127
6,1230170166614482944,Carpe diem,Carpedi92670638,None,2020-02-19 16:40:09,425,459,13101,8830
7,1193535233242664960,𝒎𝒆𝒍𝒆𝒌,meysimek,Germany,2019-11-10 14:25:56,931,976,366,475
8,4707764075,UpsidedownTurtle 🧢,Adakisn,None,2016-01-04 19:43:48,76,82,7471,6041
9,14135350,Bianna Golodryga,biannagolodryga,New York,2008-03-12 21:25:29,148430,3376,13104,23343
